# Constraint Suggestions Basic Tutorial

__Updated June 2024 to use a new dataset__

This Jupyter notebook will give a basic tutorial on how to use PyDeequ's Constraint Suggestions module.

In [1]:
import os
# indicate your Spark version, here we use Spark 3.5 with pydeequ 1.4.0
os.environ["SPARK_VERSION"] = '3.5' 

In [2]:
from pyspark.sql import SparkSession, Row, DataFrame
import json
import pandas as pd
import sagemaker_pyspark

import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.memory", "15g")
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .getOrCreate())

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-af15f371-12cc-40de-ae72-695f78861e66;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.3-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.

24/06/14 23:56:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/06/14 23:56:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/14 23:56:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/14 23:56:36 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/06/14 23:56:36 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


### We will be using the synthetic reviews dataset for Electronics products

In [3]:
df = spark.read.parquet("s3a://aws-bigdata-blog/generated_synthetic_reviews/data/product_category=Electronics/")

df.printSchema()

24/06/14 23:56:38 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: long (nullable = true)
 |-- helpful_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- insight: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- review_year: long (nullable = true)



In [4]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

print(json.dumps(suggestionResult, indent=2))

24/06/14 23:56:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


{
  "constraint_suggestions": [
    {
      "constraint_name": "ComplianceConstraint(Compliance('insight' has value range 'N', 'Y',`insight` IN ('N', 'Y'),None))",
      "column_name": "insight",
      "current_value": "Compliance: 1",
      "description": "'insight' has value range 'N', 'Y'",
      "suggesting_rule": "CategoricalRangeRule(com.amazon.deequ.suggestions.rules.CategoricalRangeRule$$$Lambda$2307/0x00000008012b0040@31a0d64e)",
      "rule_description": "If we see a categorical range for a column, we suggest an IS IN (...) constraint",
      "code_for_constraint": ".isContainedIn(\"insight\", [\"N\", \"Y\"])"
    },
    {
      "constraint_name": "CompletenessConstraint(Completeness(insight,None))",
      "column_name": "insight",
      "current_value": "Completeness: 1.0",
      "description": "'insight' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
    

### For more info ... look at full list of suggestions in `docs/suggestions.md` 